# Global Minimial Viable Products (1)

Combine:
* SMAP available porosity
* ERA5 runoff
* MERIT terrain slope
* USDA soil texture class

into global product for PRZ water restoration

In [1]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm
from earthshot import mon_stats
from statistics import mean
import folium
from folium import plugins

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [30]:
#import global MAR features

import json
with open('Mar_swales_2.json') as f: MAR_map = json.load(f)

MAR_featurecollection = ee.FeatureCollection(MAR_map)

In [4]:
#common parameters
bbox_global = ee.Geometry.BBox(-180,-90,180,90)

# SMAP-USDA Available Porosity
More available porosity is better.

In [5]:
# Sum up available porosity in mm across the year
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')
avail_porosity = (smap_usda_clim
                  .filter(ee.Filter.eq('band', 'avail_porosity_mm')))

avail_porosity_img = avail_porosity.sum()

avail_porosity_img_scaled = (norm.img_scale(
    avail_porosity_img, area_of_interest= bbox_global))

## ERA5 Runoff

In [6]:
# Sum up runoff across the year to include in product score
runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')

runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], runoff_clim)

runoff_img = ee.ImageCollection(runoff_clim_m['avgs'].get('surface_runoff')).sum().multiply(720)
runoff_mask = runoff_img.gte(0.05) #values masked to areas that runoff 50 mm/ year at least
runoff_img_masked = runoff_img.updateMask(runoff_mask)

runoff_img_scaled = (norm.img_scale(
    runoff_img, 
    area_of_interest= bbox_global))

print(norm.img_range(runoff_img_masked, area_of_interest = bbox_global))

[0.050000064074993134, 387.5152140632271]


## MERIT Terrain Slope

In [64]:
#include slope as a factor in scoring product, mask for lower slopes
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(10).And(slope_img.gt(0))
slope_img_masked = slope_img.updateMask(slope_mask).pow(-1)
slope_img_scaled = norm.img_scale(slope_img_masked, area_of_interest=bbox_global)
print(norm.img_range(runoff_img_scaled, area_of_interest = bbox_global))

[0, 1]


# Soil Types

In [8]:
#incorporate soil types as a factor in scoring for PRZ
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [9]:
#categorizing soil types and depths based on retaining plant-available water, grouping top soils and bottom soils together
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.55" +
        ": (b('b0') == 7) ? 0.55" +
        ": (b('b0') == 6) ? 0.55" +
        ": (b('b0') == 5) ? 0.55" +
        ": (b('b0') == 4) ? 0.1" +
        ": (b('b0') == 3) ? 0.1" +
        ": (b('b0') == 2) ? 0.1" +
        ": (b('b0') == 1) ? 0.1" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.55" +
        ": (b('b0') == 7) ? 0.55" +
        ": (b('b0') == 6) ? 0.55" +
        ": (b('b0') == 5) ? 0.55" +
        ": (b('b0') == 4) ? 0.1" +
        ": (b('b0') == 3) ? 0.1" +
        ": (b('b0') == 2) ? 0.1" +
        ": (b('b0') == 1) ? 0.1" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.55" +
        ": (b('b0') == 7) ? 0.55" +
        ": (b('b0') == 6) ? 0.55" +
        ": (b('b0') == 5) ? 0.55" +
        ": (b('b0') == 4) ? 0.1" +
        ": (b('b0') == 3) ? 0.1" +
        ": (b('b0') == 2) ? 0.1" +
        ": (b('b0') == 1) ? 0.1" +
        ": 0")

soil_60 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.55" +
        ": (b('b60') == 4) ? 0.55" +
        ": (b('b60') == 6) ? 0.55" +
        ": (b('b60') == 9) ? 0.55" +
        ": (b('b60') == 1) ? 0.1" +
        ": (b('b60') == 3) ? 0.1" +
        ": (b('b60') == 11) ? 0.1" +
        ": (b('b60') == 12) ? 0.1" +
        ": 0")

soil_100 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.55" +
        ": (b('b60') == 4) ? 0.55" +
        ": (b('b60') == 6) ? 0.55" +
        ": (b('b60') == 9) ? 0.55" +
        ": (b('b60') == 1) ? 0.1" +
        ": (b('b60') == 3) ? 0.1" +
        ": (b('b60') == 11) ? 0.1" +
        ": (b('b60') == 12) ? 0.1" +
        ": 0")

soil_200 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.55" +
        ": (b('b60') == 4) ? 0.55" +
        ": (b('b60') == 6) ? 0.55" +
        ": (b('b60') == 9) ? 0.55" +
        ": (b('b60') == 1) ? 0.1" +
        ": (b('b60') == 3) ? 0.1" +
        ": (b('b60') == 11) ? 0.1" +
        ": (b('b60') == 12) ? 0.1" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

bottom_soils = soil_60.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_60.select('constant'),
                                  'soil_100': soil_100.select('constant'),
                                  'soil_200': soil_200.select('constant')})

In [10]:
#scaling top soils and bottom soils
top_soils_scaled = norm.img_scale(top_soils, area_of_interest=bbox_global)
bottom_soils_scaled = norm.img_scale(bottom_soils, area_of_interest=bbox_global)

In [11]:
print(norm.img_range(top_soils_scaled, area_of_interest = bbox_global))

[0, 1]


## PRZ Score
Simply give higher scores to places that have both high runoff, available porosity, favorable soil type and slope. Masked for lower slopes.

In [74]:
score = (runoff_img_scaled.multiply(.25)
         .add(avail_porosity_img_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25))
         .add(top_soils_scaled.multiply(.125))
        .add(bottom_soils_scaled.multiply(.125))
        .updateMask(slope_mask)
        .updateMask(runoff_mask))

score_2 = (avail_porosity_img_scaled.multiply(.8)
         #.add(slope_img_scaled.multiply(.1))
        .add(runoff_img_scaled.multiply(.2))
        .updateMask(slope_mask)
        .updateMask(runoff_mask))

score = norm.img_scale(score, area_of_interest=bbox_global).rename('score')
score_2 = norm.img_scale(score_2, area_of_interest=bbox_global).rename('score_2')

In [75]:
#visualizing PRZ score, other variables

score_range = norm.img_range(score, area_of_interest = bbox_global)
palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum= score_range[0], maximum= 10)

box_corners = bbox_global.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': score_range[0], 'max': score_range[1], 'dimensions': 512,
    'palette': palette}

vis_params_runoff = {
    'min': 0, 'max': 2, 'dimensions': 512,
    'palette': palette}

vis_params_slope = {
    'min': 0, 'max': 10, 'dimensions': 512,
    'palette': palette}

vis_params_poros = {
    'min': 0, 'max': 2000, 'dimensions': 512,
    'palette': palette}


In [77]:
#creating the map, visualizing score

avail_porosity_masked = avail_porosity_img.updateMask(runoff_mask).updateMask(slope_mask)

the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=1, height=400)

the_map.add_ee_layer(score, vis_params, name = 'PRZ Score-1')
the_map.add_ee_layer(score_2, vis_params, name = 'PRZ Score-2')
the_map.add_ee_layer(top_soils_scaled, vis_params, name = 'Top Soils')
the_map.add_ee_layer(avail_porosity_masked, vis_params_poros, name = 'Available Porosity')
the_map.add_ee_layer(runoff_img_masked, vis_params_runoff, name = 'Runoff')
the_map.add_ee_layer(slope_img_masked, vis_params_slope, name = 'Slope')

the_map.add_ee_layer(MAR_featurecollection.draw(color= '006600', pointRadius = 5, strokeWidth= 5), {}, name = 'MAR Swale locations')




#the_map.add_ee_layer(slope_img_scaled, vis_params, name = 'Slope')

#the_map.add_ee_layer(top_soils_scaled, vis_params, name = 'Top Soils')

vis.folium_display(the_map)



In [ ]:
#slope does not seem to be a factor in selection...
#runoff is not well correlated
#a combination of available porosity and soil type seems to be well correlated